In [1]:
import cv2
import matplotlib.pyplot as plt
import cvlib as cv
import urllib.request
import numpy as np
from cvlib.object_detection import draw_bbox
import concurrent.futures
import torch
import cv2
import time
url='http://192.168.1.10/cam-mid.jpg'
im=None

print(f"[INFO] Loading model... ")
## loading the custom trained model
# model =  torch.hub.load('ultralytics/yolov5', 'custom', path='last.pt',force_reload=True) ## if you want to download the git repo and then run the detection
model =  torch.hub.load('D:\8th_Sem_Proj\Dataset\8th_Sem_Proj\With_YoloV5\yolov5', 'custom', source ='local', path=r'D:\8th_Sem_Proj\Dataset\8th_Sem_Proj\With_YoloV5\yolov5\runs\train\exp9\weights\best.pt',force_reload=True) ### The repo is stored locally
x = None
classes = model.names ### class names in string format
coordinates = (100,100)
font = cv2.FONT_HERSHEY_SIMPLEX
fontScale = 1
color = (255,0,255)
thickness = 2
def run1():
    x = None
    cv2.namedWindow("live transmission", cv2.WINDOW_AUTOSIZE)
    while True:
        img_resp=urllib.request.urlopen(url)
        #x = cv2.imread(bytearray(img_resp.read()),dtype=np.uint8)
        imgnp=np.array(bytearray(img_resp.read()),dtype=np.uint8)
        im = cv2.imdecode(imgnp,-1)
        results = d(im)
        results = results.xyxyn[0][:, -1]
        #print(im)
        if len(results)!=0:
            if results == 0:
                text = 'Early_Blight'
                x=0
            elif results == 1:
                text = 'Healthy'
                x=1
            elif results == 2:
                text = 'Mold_leaf'
                x=2
        if x:
            im = cv2.putText(im, text, coordinates, font, fontScale, color, thickness, cv2.LINE_AA)

        cv2.imshow('live transmission',im)
        
        key=cv2.waitKey(5)
        if key==ord('q'):
            break
        #break
    #return im
    cv2.destroyAllWindows()

def run2():
    cv2.namedWindow("detection", cv2.WINDOW_AUTOSIZE)
    while True:
        img_resp=urllib.request.urlopen(url)
        imgnp=np.array(bytearray(img_resp.read()),dtype=np.uint8)
        im = cv2.imdecode(imgnp,-1)
        #cv.imshow(im)
        bbox, label, conf = cv.detect_common_objects(im)
        # print(bbox)
        # print(label)
        # print(conf)
        im = draw_bbox(im, bbox, label, conf)

        cv2.imshow('detection',im)
        key=cv2.waitKey(5)
        if key==ord('q'):
            break
        #break

    cv2.destroyAllWindows()





[INFO] Loading model... 


YOLOv5  v6.1-177-gd059d1d torch 1.11.0 CUDA:0 (GeForce MX230, 2048MiB)

Fusing layers... 
Model summary: 213 layers, 7018216 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


In [2]:
def d(im):



        ### -------------------------------------- function to run detection ---------------------------------------------------------
        def detectx (frame, model):
            frame = [frame]
            #print(f"[INFO] Detecting. . . ")
            results = model(frame)
            # results.show()
            # print( results.xyxyn[0])
            # print(results.xyxyn[0][:, -1])
            # print(results.xyxyn[0][:, :-1])
            return results
            labels, cordinates = results.xyxyn[0][:, -1], results.xyxyn[0][:, :-1]

            return labels, cordinates

        ### ------------------------------------ to plot the BBox and results --------------------------------------------------------
        def plot_boxes(results, frame,classes):

            """
            --> This function takes results, frame and classes
            --> results: contains labels and coordinates predicted by model on the given frame
            --> classes: contains the strting labels
            """
            labels, cord = results
            n = len(labels)
            x_shape, y_shape = frame.shape[1], frame.shape[0]
            if n!=0:
                print(f"[INFO] Total {n} detections. . . ")
                print(f"[INFO] Looping through all detections. . . ")


            ### looping through the detections
            for i in range(n):
                row = cord[i]
                if row[4] >= 0.55: ### threshold value for detection. We are discarding everything below this value
                    print(f"[INFO] Extracting BBox coordinates. . . ")
                    x1, y1, x2, y2 = int(row[0]*x_shape), int(row[1]*y_shape), int(row[2]*x_shape), int(row[3]*y_shape) ## BBOx coordniates
                    text_d = classes[int(labels[i])]


                    if text_d == 'mask':
                        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2) ## BBox
                        cv2.rectangle(frame, (x1, y1-20), (x2, y1), (0, 255,0), -1) ## for text label background

                        
                        cv2.putText(frame, text_d + f" {round(float(row[4]),2)}", (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 0.7,(255,255,255), 2)

                    elif text_d == 'nomask':
                        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0,255), 2) ## BBox
                        cv2.rectangle(frame, (x1, y1-20), (x2, y1), (0, 0,255), -1) ## for text label background

                        
                        cv2.putText(frame, text_d + f" {round(float(row[4]),2)}", (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 0.7,(255,255,255), 2)
                    ## print(row[4], type(row[4]),int(row[4]), len(text_d))

            return frame




        ### ---------------------------------------------- Main function -----------------------------------------------------

        def main(img_path=None, vid_path=None,vid_out = None):



            #if img_path != None:
                #print(f"[INFO] Working with image: {img_path}")
                #frame = cv2.imread(img_path)
            frame = cv2.cvtColor(img_path,cv2.COLOR_BGR2RGB)
                
            results = detectx(frame, model = model) ### DETECTION HAPPENING HERE    

            #print('resulta/////////\t',results)
            #return results
            #frame = cv2.cvtColor(frame,cv2.COLOR_RGB2BGR)
            #frame = plot_boxes(results, frame,classes = classes)
            return results
            #     cv2.namedWindow("img_only", cv2.WINDOW_NORMAL) ## creating a free windown to show the result

            #     while True:
            #         # frame = cv2.cvtColor(frame,cv2.COLOR_RGB2BGR)

            #         cv2.imshow("img_only", frame)

            #         if cv2.waitKey(5) & 0xFF == 27:
            #             print(f"[INFO] Exiting. . . ")
            #             cv2.imwrite("final_output.jpg",frame) ## if you want to save he output result.

            #             break

            # elif vid_path !=None:
            #     print(f"[INFO] Working with video: {vid_path}")

            #     ## reading the video
            #     cap = cv2.VideoCapture(vid_path)


            #     if vid_out: ### creating the video writer if video output path is given

            #         # by default VideoCapture returns float instead of int
            #         width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
            #         height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
            #         fps = int(cap.get(cv2.CAP_PROP_FPS))
            #         codec = cv2.VideoWriter_fourcc(*'mp4v') ##(*'XVID')
            #         out = cv2.VideoWriter(vid_out, codec, fps, (width, height))

            #     # assert cap.isOpened()
            #     frame_no = 1

            #     cv2.namedWindow("vid_out", cv2.WINDOW_NORMAL)
            #     while True:
            #         # start_time = time.time()
            #         ret, frame = cap.read()
            #         if ret :
            #             print(f"[INFO] Working with frame {frame_no} ")

            #             frame = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
            #             results = detectx(frame, model = model)
            #             frame = cv2.cvtColor(frame,cv2.COLOR_RGB2BGR)
            #             frame = plot_boxes(results, frame,classes = classes)
                        
            #             cv2.imshow("vid_out", frame)
            #             if vid_out:
            #                 print(f"[INFO] Saving output video. . . ")
            #                 out.write(frame)

            #             if cv2.waitKey(5) & 0xFF == 27:
            #                 break
            #             frame_no += 1
                
            #     print(f"[INFO] Clening up. . . ")
            #     ### releaseing the writer
            #     out.release()
                
            #     ## closing all windows
            #     cv2.destroyAllWindows()



        ### -------------------  calling the main function-------------------------------


        # main(vid_path="facemask.mp4",vid_out="facemask_result.mp4") ### for custom video
        # main(vid_path=0,vid_out="webcam_facemask_result.mp4") #### for webcam

        #print('main')
        return main(im)
        #main(img_path="crowd_mask181.jpg") ## for image

In [3]:
classes

['Early_Blight', 'Healthy', 'mold_leaf']

In [4]:
if __name__ == '__main__':
    print("started")
    with concurrent.futures.ProcessPoolExecutor() as executer:
            run1()
            #run1()
            #f1= executer.submit(run1)
            #f2= executer.submit(run2)
            #print(results)

started


RuntimeError: Boolean value of Tensor with more than one value is ambiguous

: 

In [23]:
x = results.xyxyn[0][:, -1][0]
x == 1

tensor(True, device='cuda:0')

In [26]:
if x==0:
    print(1)

In [28]:
labels, cord = results
n = len(labels)
print(labels)
print(cord)
print(n)
#x_shape, y_shape = frame.shape[1], frame.shape[0]

tensor([2.], device='cuda:0')
tensor([[0.17168, 0.24908, 0.47312, 0.50295, 0.27142]], device='cuda:0')
1
